<a href="https://colab.research.google.com/github/LuSant22/Capstone_1/blob/master/LA%20and%20SD%20ramen%20restaurant%20analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ramen Restaurant Analysis between San Diego and LA
## (Best Location)

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

/bin/bash: conda: command not found
Libraries imported.


In [2]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604'
limit = 200
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: H3M1VJS1APMMUTEK4020EN11QDC03X1QZXK2AQRPI21WCMC5
CLIENT_SECRET:ZN4V5WZSMWU4KB1T4TE01F5ZGNE3QWCY4XMA2KEO0KPYE5TB


### Choose the location

In [3]:
address = 'San Diego, California'

geolocator = Nominatim(user_agent="To_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Diego are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of San Diego are 32.7174209, -117.1627714.


In [53]:
search_query = 'Ramen'
radius = 5000
print(search_query + ' .... OK!')

Ramen .... OK!


In [54]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, limit)
url

'https://api.foursquare.com/v2/venues/search?client_id=H3M1VJS1APMMUTEK4020EN11QDC03X1QZXK2AQRPI21WCMC5&client_secret=ZN4V5WZSMWU4KB1T4TE01F5ZGNE3QWCY4XMA2KEO0KPYE5TB&ll=32.7174209,-117.1627714&v=20180604&query=Ramen&radius=5000&limit=200'

In [0]:
results = requests.get(url).json()


In [56]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,57d89409498e1e9f50b759a9,BeShock Ramen & Sake Bar,"[{'id': '55a59bace4b013909087cb24', 'name': 'R...",v-1581903409,False,1288 Market St,at 13th St,32.711710,-117.153028,"[{'label': 'display', 'lat': 32.71171029471891...",1112,92101,US,San Diego,CA,United States,"[1288 Market St (at 13th St), San Diego, CA 92...",1599832,https://www.grubhub.com/restaurant/beshock-ram...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png
1,564ff470498ee3d910f427e5,Ramen Yamadaya,"[{'id': '55a59bace4b013909087cb24', 'name': 'R...",v-1581903409,False,531 Broadway,5th Ave,32.715721,-117.159623,"[{'label': 'display', 'lat': 32.71572136624164...",350,92101,US,San Diego,CA,United States,"[531 Broadway (5th Ave), San Diego, CA 92101, ...",NaN,NaN,NaN,NaN,NaN,NaN
2,5680b03a498e78a8fe2287dc,Tajima Ramen East Village,"[{'id': '55a59bace4b013909087cb24', 'name': 'R...",v-1581903409,False,901 E St,NaN,32.714553,-117.156398,"[{'label': 'display', 'lat': 32.71455263434286...",676,92101,US,San Diego,CA,United States,"[901 E St, San Diego, CA 92101, United States]",NaN,NaN,NaN,NaN,NaN,NaN
3,5c6f77ca1af852002cdef576,Ramen Yamachan,"[{'id': '55a59bace4b013909087cb24', 'name': 'R...",v-1581903409,False,NaN,NaN,32.715587,-117.159664,"[{'label': 'display', 'lat': 32.715587, 'lng':...",355,92101,US,San Diego,CA,United States,"[San Diego, CA 92101, United States]",NaN,NaN,NaN,NaN,NaN,NaN
4,58f12c3a780eee43495e9721,Hachi Ramen,"[{'id': '55a59bace4b013909087cb24', 'name': 'R...",v-1581903409,False,2505 5th Ave,Laurel,32.731535,-117.160256,"[{'label': 'display', 'lat': 32.73153490575097...",1588,92103,US,San Diego,CA,United States,"[2505 5th Ave (Laurel), San Diego, CA 92103, U...",1599393,https://www.grubhub.com/restaurant/hachi-ramen...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png


## As we can see in San Diego there are only 13 Ramen restaurants in a radius of 5000 

In [58]:
dataframe_filtered.name

0      BeShock Ramen & Sake Bar
1                Ramen Yamadaya
2     Tajima Ramen East Village
3                Ramen Yamachan
4                   Hachi Ramen
5     Rakiraki Ramen & Tsukemen
6                   Ramen Ryoma
7          HiroNori Craft Ramen
8              Tajima Ramen Bar
9                 Nishiki Ramen
10             Tajima Ramen Bar
11                 Manten Ramen
12    Rakiraki Ramen & Tsukemen
13                  Tokyo Ramen
Name: name, dtype: object

In [59]:
address_2 = 'Los Angeles, California'

geolocator_2 = Nominatim(user_agent="To_explorer")
location_2 = geolocator.geocode(address_2)
latitude_2  = location_2.latitude
longitude_2  = location_2.longitude
print('The geograpical coordinate of LA are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of LA are 32.7174209, -117.1627714.


In [60]:
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_2, longitude_2, VERSION, search_query, radius, limit)
url2

'https://api.foursquare.com/v2/venues/search?client_id=H3M1VJS1APMMUTEK4020EN11QDC03X1QZXK2AQRPI21WCMC5&client_secret=ZN4V5WZSMWU4KB1T4TE01F5ZGNE3QWCY4XMA2KEO0KPYE5TB&ll=34.0536909,-118.2427666&v=20180604&query=Ramen&radius=5000&limit=200'

In [0]:
results2 = requests.get(url2).json()

In [0]:
venues2= results2['response']['venues']

# tranform venues into a dataframe
dataframe2 = json_normalize(venues2)


In [63]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns2 = ['name', 'categories'] + [col for col in dataframe2.columns if col.startswith('location.')] + ['id']
dataframe_filtered2 = dataframe2.loc[:, filtered_columns]

# function that extracts the category of the venue

# filter the category for each row
dataframe_filtered2['categories'] = dataframe_filtered2.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered2.columns = [column.split('.')[-1] for column in dataframe_filtered2.columns]

dataframe_filtered2

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Orochon Ramen,Noodle House,123 Astronaut E S Onizuka St #303,at San Pedro St,34.049939,-118.242319,"[{'label': 'display', 'lat': 34.04993935721044...",419,90012,US,Los Angeles,CA,United States,[123 Astronaut E S Onizuka St #303 (at San Ped...,46ddce98f964a520934a1fe3
1,Mr. Ramen,Ramen Restaurant,341 1/2 E 1st St,btwn S Central & S San Pedro St,34.049630,-118.239822,"[{'label': 'display', 'lat': 34.04962989405902...",527,90012,US,Los Angeles,CA,United States,[341 1/2 E 1st St (btwn S Central & S San Pedr...,4a48599df964a520b4aa1fe3
2,My Ramen Bar,Noodle House,321 E 1st St,NaN,34.049993,-118.240341,"[{'label': 'display', 'lat': 34.04999288080512...",468,90012,US,Los Angeles,CA,United States,"[321 E 1st St, Los Angeles, CA 90012, United S...",54aae895498e545686bde596
3,Ramen Maruya,Asian Restaurant,104 Japanese Village Plaza Mall,NaN,34.049188,-118.239742,"[{'label': 'display', 'lat': 34.04918755373025...",573,90012,US,Los Angeles,CA,United States,"[104 Japanese Village Plaza Mall, Los Angeles,...",53c9ed2c498e889ccf25472e
4,Rakkan Ramen,Ramen Restaurant,359 E 1st St,Central,34.049447,-118.239528,"[{'label': 'display', 'lat': 34.04944704252609...",558,90012,US,Los Angeles,CA,United States,"[359 E 1st St (Central), Los Angeles, CA 90012...",596448501987ec3cfd0626f0
5,Ramen Hood,Ramen Restaurant,317 S Broadway,NaN,34.050631,-118.248688,"[{'label': 'display', 'lat': 34.05063099262521...",643,90013,US,Los Angeles,CA,United States,"[317 S Broadway, Los Angeles, CA 90013, United...",563be5c0cd107fbf879eceb6
6,Hachioji Craft Ramen,Ramen Restaurant,313 E 1st St,NaN,34.050106,-118.240510,"[{'label': 'display', 'lat': 34.050106, 'lng':...",450,90012,US,Los Angeles,CA,United States,"[313 E 1st St, Los Angeles, CA 90012, United S...",5af2b349491be700392d81f1
7,Silver Lake Ramen,Ramen Restaurant,2927 W Sunset Blvd,NaN,34.083438,-118.273655,"[{'label': 'display', 'lat': 34.08343777811164...",4367,90026,US,Los Angeles,CA,United States,"[2927 W Sunset Blvd, Los Angeles, CA 90026, Un...",50611458e4b06bd054509217
8,Men Oh Tokushima Ramen,Ramen Restaurant,457 E 2nd St,NaN,34.047238,-118.238668,"[{'label': 'display', 'lat': 34.047238, 'lng':...",811,90012,US,Los Angeles,CA,United States,"[457 E 2nd St, Los Angeles, CA 90012, United S...",506b47e3e4b010161f10b560
9,Silver Lake Ramen,Ramen Restaurant,"615 South Spring St. Los Angeles, CA 90014",NaN,34.045487,-118.251445,"[{'label': 'display', 'lat': 34.045487, 'lng':...",1214,90014,US,Los Angeles,CA,United States,"[615 South Spring St. Los Angeles, CA 90014, ...",5c8c00f46336be0037fd65ee


## As we can see there are 31 Ramen restaurants in LA in a radius of 5000

In [70]:
dataframe_filtered2.name

0                   Orochon Ramen
1                       Mr. Ramen
2                    My Ramen Bar
3                    Ramen Maruya
4                    Rakkan Ramen
5                      Ramen Hood
6            Hachioji Craft Ramen
7               Silver Lake Ramen
8          Men Oh Tokushima Ramen
9               Silver Lake Ramen
10                Jinya Ramen Bar
11                     DTLA Ramen
12                      Ramen Koo
13                    Ramen Champ
14              Hashi Ramen House
15                      Daikokuya
16                  Shin-Sen-Gumi
17                Hakata Ikkousha
18               9th Street Ramen
19                Ramen Tatsunoya
20                        Kouraku
21                    Ramen Kenjo
22        Silver Lake Ramen To-Go
23               Silverlake Ramen
24                      Ten Ramen
25                    Ginza Ramen
26    Chashu Ramen & Bowl Izakaya
27                   Momota Ramen
28                    Sheng Ramen
29            

In [72]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map at the center of SD
folium.vector_layers.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Center',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add blue circle markers to Ramen Restaurants in SD
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.vector_layers.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [57]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,BeShock Ramen & Sake Bar,Ramen Restaurant,1288 Market St,at 13th St,32.711710,-117.153028,"[{'label': 'display', 'lat': 32.71171029471891...",1112,92101,US,San Diego,CA,United States,"[1288 Market St (at 13th St), San Diego, CA 92...",57d89409498e1e9f50b759a9
1,Ramen Yamadaya,Ramen Restaurant,531 Broadway,5th Ave,32.715721,-117.159623,"[{'label': 'display', 'lat': 32.71572136624164...",350,92101,US,San Diego,CA,United States,"[531 Broadway (5th Ave), San Diego, CA 92101, ...",564ff470498ee3d910f427e5
2,Tajima Ramen East Village,Ramen Restaurant,901 E St,NaN,32.714553,-117.156398,"[{'label': 'display', 'lat': 32.71455263434286...",676,92101,US,San Diego,CA,United States,"[901 E St, San Diego, CA 92101, United States]",5680b03a498e78a8fe2287dc
3,Ramen Yamachan,Ramen Restaurant,NaN,NaN,32.715587,-117.159664,"[{'label': 'display', 'lat': 32.715587, 'lng':...",355,92101,US,San Diego,CA,United States,"[San Diego, CA 92101, United States]",5c6f77ca1af852002cdef576
4,Hachi Ramen,Ramen Restaurant,2505 5th Ave,Laurel,32.731535,-117.160256,"[{'label': 'display', 'lat': 32.73153490575097...",1588,92103,US,San Diego,CA,United States,"[2505 5th Ave (Laurel), San Diego, CA 92103, U...",58f12c3a780eee43495e9721
5,Rakiraki Ramen & Tsukemen,Ramen Restaurant,NaN,NaN,32.728193,-117.170418,"[{'label': 'display', 'lat': 32.728193, 'lng':...",1396,92101,US,San Diego,CA,United States,"[San Diego, CA 92101, United States]",5c745d97c62b49002cc52577
6,Ramen Ryoma,Ramen Restaurant,NaN,NaN,32.748578,-117.156873,"[{'label': 'display', 'lat': 32.748578, 'lng':...",3512,92103,US,San Diego,CA,United States,"[San Diego, CA 92103, United States]",5df198c149c1880008ca6e9b
7,HiroNori Craft Ramen,Ramen Restaurant,3803 5th Ave,NaN,32.747153,-117.160272,"[{'label': 'display', 'lat': 32.747153, 'lng':...",3318,92103,US,San Diego,CA,United States,"[3803 5th Ave, San Diego, CA 92103, United Sta...",5c649c92829b0c002cb61a68
8,Tajima Ramen Bar,Noodle House,3739 6th Ave,NaN,32.746016,-117.159451,"[{'label': 'display', 'lat': 32.74601618653234...",3198,92103,US,San Diego,CA,United States,"[3739 6th Ave, San Diego, CA 92103, United Sta...",53c5f7d7498e6ff03800eaee
9,Nishiki Ramen,Ramen Restaurant,1040 University Ave,NaN,32.748653,-117.154480,"[{'label': 'display', 'lat': 32.748653, 'lng':...",3562,92103,US,San Diego,CA,United States,"[1040 University Ave, San Diego, CA 92103, Uni...",5b42be98d69ed0002c2ae3ab


In [73]:
# add blue circle markers to Ramen Restaurants in SD
for lat, lng, label in zip(dataframe_filtered2.lat, dataframe_filtered2.lng, dataframe_filtered2.categories):
    folium.vector_layers.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
# display map
venues_map